In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/adultcensus/adult-census.csv
/kaggle/input/adultcensus/adult-census-numeric-test.csv


In [2]:
import pandas as pd

adult_census = pd.read_csv("../input/adultcensus/adult-census.csv")

target_name = "class"
numerical_columns = [
    "age", "capital-gain", "capital-loss", "hours-per-week"]

target = adult_census[target_name]
data = adult_census[numerical_columns]

In [3]:
data.head()

,age,capital-gain,capital-loss,hours-per-week
0,25,0,0,40
1,38,0,0,50
2,28,0,0,40
3,44,7688,0,40
4,18,0,0,30


In [4]:
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

model = Pipeline(steps=[
    ("preprocessor", StandardScaler()),
    ("Classifier", LogisticRegression())
])

In [5]:
from sklearn import set_config
set_config(display='diagram')
model

Pipeline(steps=[('preprocessor', StandardScaler()),
                ('Classifier', LogisticRegression())])

* We will evaluate the generalization performance of the model using `sklean.model_selection.cross_validate`

In [6]:
from sklearn.model_selection import cross_validate

cv_results = cross_validate(model, data, target)
scores = cv_results["test_score"]
print(f"Accuracy score via cross-validation:\n"
      f"{scores.mean():.3f} +/- {scores.std():.3f}")

Accuracy score via cross-validation:
0.800 +/- 0.003


We created a model with the default `C` value that is equal to 1. If we
wanted to use a different `C` parameter we could have done so when we created
the `LogisticRegression` object with something like `LogisticRegression(C=1e-3)`.


We can also change the parameter of a model after it has been created with
the `set_params` method, which is available for all scikit-learn estimators.
For example, we can set `C=1e-3`, fit and evaluate the model:

In [7]:
model.set_params(Classifier__C=1e-3)
cv_results = cross_validate(model, data, target)
scores = cv_results["test_score"]
print(f"Accuracy score via cross-validation:\n"
      f"{scores.mean():.3f} +/- {scores.std():.3f}")

Accuracy score via cross-validation:
0.787 +/- 0.002


When the model of interest is a `Pipeline`, the parameter names are of the
form `<model_name>__<parameter_name>` (note the double underscore in the
middle). In our case, `classifier` comes from the `Pipeline` definition and
`C` is the parameter name of `LogisticRegression`.

In general, you can use the `get_params` method on scikit-learn models to
list all the parameters with their values. For example, if you want to
get all the parameter names, you can use:

In [8]:
for parameter in model.get_params():
    print(parameter)

memory
steps
verbose
preprocessor
Classifier
preprocessor__copy
preprocessor__with_mean
preprocessor__with_std
Classifier__C
Classifier__class_weight
Classifier__dual
Classifier__fit_intercept
Classifier__intercept_scaling
Classifier__l1_ratio
Classifier__max_iter
Classifier__multi_class
Classifier__n_jobs
Classifier__penalty
Classifier__random_state
Classifier__solver
Classifier__tol
Classifier__verbose
Classifier__warm_start


`.get_params()` returns a `dict` whose keys are the parameter names and whose
values are the parameter values. If you want to get the value of a single
parameter, for example `classifier__C`, you can use:

In [9]:
model.get_params()['Classifier__C']

0.001

In [11]:
for C in [1e-3, 1e-2, 1e-1, 1, 10]:
    model.set_params(Classifier__C=C)
    cv_results = cross_validate(model, data, target)
    scores = cv_results["test_score"]
    print(f"Accuracy score via cross-validation with C={C}:\n"
          f"{scores.mean():.3f} +/- {scores.std():.3f}")

Accuracy score via cross-validation with C=0.001:
0.787 +/- 0.002
Accuracy score via cross-validation with C=0.01:
0.799 +/- 0.003
Accuracy score via cross-validation with C=0.1:
0.800 +/- 0.003
Accuracy score via cross-validation with C=1:
0.800 +/- 0.003
Accuracy score via cross-validation with C=10:
0.800 +/- 0.003


We can see that as long as C is high enough, the model seems to perform
well.

What we did here is very manual: it involves scanning the values for C
and picking the best one manually. In the next lesson, we will see how
to do this automatically.
